In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from tensorboardX import SummaryWriter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import time
import os

In [7]:
class AML(nn.Module):
    def __init__(self,D_in, D_out):
        super(AML, self).__init__()
        self.l1=nn.Linear(D_in,20)
        self.l2=nn.Linear(20,D_out)
        self.sig=nn.Sigmoid()
        
    def forward(self, x):
        x=self.l1(x)
        x=F.relu(x)
        x=self.l2(x)
        x=self.sig(x)
        return x
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [8]:
model=AML(9,2)
a=torch.ones(9)
model(a)

tensor([0.4587, 0.5507], grad_fn=<SigmoidBackward>)

In [10]:
load_data = torch.load('../data.pt')
X = load_data["X"]
y = load_data["y"]
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)
loader = TensorDataset(torch.Tensor(X_train).to(device), torch.Tensor(y_train).to(device))
data = DataLoader(loader, batch_size=128, shuffle=True)
model=model.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
loader = TensorDataset(X, y)
x=X_test[0:127].to(device)
y=y_test[0:127].to(device)
lr=np.linspace(1e-8,10,1e6)
loss=[]
for i in lr:
    y_pred = model(x)
    loss = criterion(y_pred, y.long())
    loss.append((i,))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
loader = TensorDataset(X, y)
data=DataLoader(loader, batch_size=128, shuffle=True)
for t in range(2):
    for X, y in data:
        y_pred = model(X)
        print(y.dtype, y_pred.dtype)
        loss = criterion(y_pred, y.long())
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()